# Exploring the Czech-Slovak Olympiad problem.

Our goal is to explore the solution of Achal Kumar. Specifically, we will start with all axioms and the constants $m$ and $n$ and then look for lemmas. This is a variant where we include $m * n$ as a generator.

Recall the problem:

Let ⋆ be a binary operation on a nonempty set $M$. That is, every pair $(a,b) \in  M$ is assigned an element $a$ ⋆$ b$ in $M$. Suppose that ⋆ has the additional property that $(a $ ⋆ $b) $ ⋆$ b= a$ and $a$ ⋆ $(a$ ⋆$ b)= b$ for all $a,b \in  M$.
Show that $a$ ⋆ $b = b$ ⋆ $a$ for all $a,b \in  M$.

We should derive the following lemmas.

1. $ m = (m*n)*n $ 
2. $ n = (m*n)*((m*n)*n) $ 
3. $ (m*n)*m = (m*n)*((m*n)*n) $
4. $ ((m*n)*m))*m = m*n $ 
5. $ (m*n)*m = n $
6. $ ((m*n)*m)*m = n*m $

Finally, we should get the desired result.

$ m*n  = n*m $

In [1]:
import $cp.bin.`provingground-core-jvm-6b59061b0d.fat.jar`
import provingground._ , interface._, HoTT._, learning._ 
repl.pprinter() = {
  val p = repl.pprinter()
  p.copy(
    additionalHandlers = p.additionalHandlers.orElse {
      translation.FansiShow.fansiHandler
    }
  )
}

import $cp.$                                              

import provingground._ , interface._, HoTT._, learning._ 


In [2]:
val M = "M" :: Type

val eqM = "eqM" :: M ->: M ->: Type

val a = "a" :: M
val b = "b" :: M
val c = "c" :: M

val m = "m" :: M

val n = "n" :: M

val mul = "mul" :: M ->: M ->: M

import FineDeducer.unif

M: Typ[Term] = M
eqM: Func[Term, Func[Term, Typ[Term]]] = eqM
a: Term = a
b: Term = b
c: Term = c
m: Term = m
n: Term = n
mul: Func[Term, Func[Term, Term]] = mul
import FineDeducer.unif

In [3]:
val Thm = eqM(mul(m)(n))(mul(n)(m))

val Lemma1 = eqM(m)(mul(mul(m)(n))(n))
val Lemma2 = eqM(n)(mul(mul(m)(n))(mul(mul(m)(n))(n)))
val Lemma3 = eqM(mul(mul(m)(n))(m))(mul(mul(m)(n))(mul(mul(m)(n))(n)))
val Lemma4 = eqM(mul(mul(mul(m)(n))(m))(m))(mul(m)(n))
val Lemma5 = eqM(mul(mul(m)(n))(m))(n)
val Lemma6 = eqM(mul(mul(mul(m)(n))(m))(m))(mul(n)(m))

Thm: Typ[Term] = eqM(mul(m)(n))(mul(n)(m))
Lemma1: Typ[Term] = eqM(m)(mul(mul(m)(n))(n))
Lemma2: Typ[Term] = eqM(n)(mul(mul(m)(n))(mul(mul(m)(n))(n)))
Lemma3: Typ[Term] = eqM(mul(mul(m)(n))(m))(mul(mul(m)(n))(mul(mul(m)(n))(n)))
Lemma4: Typ[Term] = eqM(mul(mul(mul(m)(n))(m))(m))(mul(m)(n))
Lemma5: Typ[Term] = eqM(mul(mul(m)(n))(m))(n)
Lemma6: Typ[Term] = eqM(mul(mul(mul(m)(n))(m))(m))(mul(n)(m))

Note that the above are defined just by using copy-paste (this is why we changed `op` to `mul`). We set up a local prover with all the axioms we use.  

In [4]:
val fullTerms : FiniteDistribution[Term] = (unif(a,b,c)(m,n, mul(m)(n), mul, eqM)(
    eqM(a)(a),
    eqM(a)(b) ->: eqM(b)(a),
    eqM(a)(b) ->: eqM(b)(c) ->: eqM(a)(c),
    eqM(mul(mul(a)(b))(b))(a),
    eqM(mul(a)(mul(a)(b)))(b),
    eqM(b)(c) ->: eqM(mul(b)(a))(mul(c)(a))
  ) * 0.5 ++ (FiniteDistribution.unif(eqM: Term) * 0.125) ++ (FiniteDistribution.unif(mul : Term ) * 0.375)).filter((t) => !Set(a, b, c).contains(t)).normalized()

fullTerms: FiniteDistribution[Term] = FiniteDistribution(
  Vector(
    Weighted(m, 0.04),
    Weighted(n, 0.04),
    Weighted(mul(m)(n), 0.04),
    Weighted(mul, 0.04),
    Weighted(eqM, 0.04),
    Weighted(axiom_{eqM(a)(a)}, 0.04),
    Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.04),
    Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.04),
    Weighted(axiom_{eqM(mul(mul(a)(b))(b))(a)}, 0.04),
    Weighted(axiom_{eqM(mul(a)(mul(a)(b)))(b)}, 0.04),
    Weighted(axiom_{(eqM(b)(c) \to eqM(mul(b)(a))(mul(c)(a)))}, 0.04),
    Weighted(eqM, 0.14),
    Weighted(mul, 0.42000000000000004)
  )
)

In [5]:
import monix.execution.Scheduler.Implicits.global
val ts = TermState(fullTerms,fullTerms.map(_.typ))
val lp0 = LocalProver(ts).addGoals(Lemma1 -> 0.1, Lemma2 -> 0.1, Lemma3 -> 0.1, Lemma4 -> 0.1, Lemma5 -> 0.1, Lemma6 -> 0.1, Thm -> 0.4).noIsles
def lp(n: Int) = lp0.sharpen(math.pow(2, n))

import monix.execution.Scheduler.Implicits.global

ts: TermState = TermState(
  FiniteDistribution(
    Vector(
      Weighted(m, 0.04),
      Weighted(n, 0.04),
      Weighted(mul(m)(n), 0.04),
      Weighted(mul, 0.04),
      Weighted(eqM, 0.04),
      Weighted(axiom_{eqM(a)(a)}, 0.04),
      Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.04),
      Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.04),
      Weighted(axiom_{eqM(mul(mul(a)(b))(b))(a)}, 0.04),
      Weighted(axiom_{eqM(mul(a)(mul(a)(b)))(b)}, 0.04),
      Weighted(axiom_{(eqM(b)(c) \to eqM(mul(b)(a))(mul(c)(a)))}, 0.04),
      Weighted(eqM, 0.14),
      Weighted(mul, 0.42000000000000004)
    )
  ),
  FiniteDistribution(
    Vector(
      Weighted(M, 0.04),
      Weighted(M, 0.04),
      Weighted(M, 0.04),
      Weighted((M → (M → M)), 0.04),
      Weighted((M → (M → 𝒰 )), 0.04),
      Weighted(∏(a : M){ eqM(a)(a) }, 0.04),
      Weighted(∏(a : M){ ∏(b : M){ (eqM(a)(b) → eqM(b)(a)) } }, 0.04),
      Weighted

In [6]:
def nslp(n: Int) = {
    val lpc = lp(n)
     lpc.nextState.map{ns => (n, ns, lpc)}
}
val bT = Utils.bestTask((1 to 30).map(nslp)).memoize

defined function nslp
bT: monix.eval.Task[Option[(Int, TermState, LocalProverStep)]] = Async(
  <function2>,
  false,
  true,
  true
)

We have set up a task that refines up to timeout. We will run this asynchronously, and then use the final state.

In [7]:
import monix.execution.Scheduler.Implicits.global
val bF = bT.runToFuture

import monix.execution.Scheduler.Implicits.global

bF: monix.execution.CancelableFuture[Option[(Int, TermState, LocalProverStep)]] = Success(
  Some(
    (
      5,
      TermState(
        FiniteDistribution(
          Vector(
            Weighted(mul(mul(m)(mul(mul(m)(n))(n))), 5.724358505097768E-4),
            Weighted(
              ($byfgytv : M) ↦ axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(mul(mul(m)(n))(mul(mul(m)(n))(n)))(n)($byfgytv)(axiom_{eqM(mul(a)(mul(a)(b)))(b)}(mul(m)(n))(n)),
              5.756091585137479E-6
            ),
            Weighted(
              axiom_{(eqM(b)(c) \to eqM(mul(b)(a))(mul(c)(a)))}(n)(mul(n)(n))(mul(n)(m)),
              1.7619793434858902E-7
            ),
            Weighted(
              mul(mul(n)(mul(m)(n)))(mul(n)(mul(n)(m))),
              1.0776276619631233E-5
            ),
            Weighted(
              mul(mul(mul(m)(n))(mul(m)(n)))(mul(m)(mul(m)(n))),
              5.944860133286563E-5
            ),
            Weighted(
              axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(m)(mul(m)(n))(mul(mul(m)(n))(mul(n)(n))),
              3.247484141009484E-7
            ),
            Weighted(
              axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(mul(mul(m)(n))(m))(mul(mul(m)(n))(mul(m)(mul(m)(n)))),
              5.695826910529936E-7
            ),
            Weighted(
              axiom_{(eqM(a)(b) \to eqM(b)(a))}(mul(mul(m)(n))(mul(m)(n)))(mul(m...

We should look for the above instantiation at least.

In [8]:
val nsT = bT.map(_.get._2).memoize
val lpT = bT.map(_.get._3).memoize

nsT: monix.eval.Task[TermState] = Async(<function2>, false, true, true)
lpT: monix.eval.Task[LocalProverStep] = Async(<function2>, false, true, true)

In [10]:
bF.value

res9: Option[scala.util.Try[Option[(Int, TermState, LocalProverStep)]]] = None

In [11]:
nsT.map(_.successes).runToFuture

res10: monix.execution.CancelableFuture[Vector[(Typ[Term], Double, FiniteDistribution[Term])]] = Success(
  Vector(
    (
      eqM(m)(mul(mul(m)(n))(n)),
      0.1,
      FiniteDistribution(
        Vector(
          Weighted(
            axiom_{(eqM(a)(b) \to eqM(b)(a))}(mul(mul(m)(n))(n))(m)(axiom_{eqM(mul(mul(a)(b))(b))(a)}(m)(n)),
            5.858437790049046E-6
          )
        )
      )
    ),
    (
      eqM(n)(mul(mul(m)(n))(mul(mul(m)(n))(n))),
      0.1,
      FiniteDistribution(
        Vector(
          Weighted(
            axiom_{(eqM(a)(b) \to eqM(b)(a))}(mul(mul(m)(n))(mul(mul(m)(n))(n)))(n)(axiom_{eqM(mul(a)(mul(a)(b)))(b)}(mul(m)(n))(n)),
            5.756091585137479E-6
          )
        )
      )
    ),
    (
      eqM(mul(mul(mul(m)(n))(m))(m))(mul(m)(n)),
      0.1,
      FiniteDistribution(
        Vector(
          Weighted(
            axiom_{eqM(mul(mul(a)(b))(b))(a)}(mul(m)(n))(m),
            1.5066270345124637E-4
          )
        )
      )
...

In [12]:
nsT.map(_.successes.map(_._1)).runToFuture

res11: monix.execution.CancelableFuture[Vector[Typ[Term]]] = Success(
  Vector(
    eqM(m)(mul(mul(m)(n))(n)),
    eqM(n)(mul(mul(m)(n))(mul(mul(m)(n))(n))),
    eqM(mul(mul(mul(m)(n))(m))(m))(mul(m)(n))
  )
)

## Results

We see that three of the lemmas have been proved:

- $m = (m * n) * n$ (Lemma 1)
- $n = (m * n) * ((m * n) * n)$ (Lemma 2)
- $((m * n) * m) * m = m * n$ (Lemma 4)

These are all instantiations, and indeed the ones we want. We should re-run with:

- recording also the `LocalProver` used.
- success when three lemmas are proved.